<a href="https://colab.research.google.com/github/benjuarez8/CS81C-Research-Bolivia/blob/main/cs81c_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
pip install Unidecode

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from unidecode import unidecode

#Load Data

In [3]:
training_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/training_data.csv")
training_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )
training_data.head()

,ID,Department,Turnout,MAS.vote,Fraud.type
0,1,Chuquisaca,0.859405,0.486528,Clean
1,2,Chuquisaca,0.894904,0.403457,Clean
2,3,Chuquisaca,0.897040,0.359153,Clean
3,4,Chuquisaca,0.904918,0.848769,VS
4,5,Chuquisaca,0.878844,0.428678,Clean


In [4]:
testing_data = pd.read_csv("https://raw.githubusercontent.com/benjuarez8/Bolivia-SURF-2020/master/CS81C/testing_data.csv")
testing_data.rename( columns={"Unnamed: 0":"ID"}, inplace=True )
testing_data.head()

,ID,Department,Municipality,Precinct,Turnout,MAS.vote
0,1,Chuquisaca,Sucre,Escuela Seccional De Chaunaca,0.863,0.789
1,2,Chuquisaca,Sucre,Unidad Educativa Chuqui Chuqui,0.869,0.702
2,3,Chuquisaca,Sucre,Unidad Educativa Chuqui Chuqui,0.892,0.686
3,4,Chuquisaca,Sucre,Escuela Fray Gabriel Maria Landini,0.822,0.778
4,5,Chuquisaca,Sucre,Escuela Fray Gabriel Maria Landini,0.847,0.729


#Data Processing

In [5]:
training_data["Fraud_num"] = [ -1 if type == "VS" else 1 if type == "BBS" else 0 for type in training_data["Fraud.type"] ]

In [6]:
training_departments = training_data["Department"].str.get_dummies()
training_data = pd.concat([training_data, training_departments], axis=1)
testing_departments = testing_data["Department"].str.get_dummies()
testing_data = pd.concat([testing_data, testing_departments], axis=1)
testing_data.rename( columns={"Potosí":unidecode("Potosí")}, inplace=True)

In [7]:
train_labels = training_data["Fraud_num"]
train = training_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]
test = testing_data[["Beni", "Chuquisaca", "Cochabamba", "La Paz", "Oruro", "Pando", "Potosi", "Santa Cruz", "Tarija", "Turnout", "MAS.vote"]]

In [8]:
X_train = np.array(train)
y_train = np.array(train_labels)
X_test = np.array(test)

#Logistic Regression

In [9]:
seed = 8
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
print(model)

LogisticRegression(max_iter=1000)


In [10]:
y_pred = model.predict(X_test)

In [11]:
test["Pred_Fraud_num"] = y_pred.tolist()
test["Pred_Fraud_type"] = [ "VS" if type == -1 else "BBS" if type == 1 else "Clean" for type in test["Pred_Fraud_num"] ]
test["Department"] = testing_data["Department"]
test["Precinct"] = testing_data["Precinct"]
test["Municipality"] = testing_data["Municipality"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [12]:
test.to_csv("log_results.csv", index=False)

#Results

In [13]:
col_names = ["Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
results = pd.DataFrame(columns = col_names)
counts = test.Pred_Fraud_type.value_counts()
clean = counts[0] / (counts[0] + counts[1] + counts[2])
tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
bbs = counts[1] / (counts[0] + counts[1] + counts[2])
vs = counts[2] / (counts[0] + counts[1] + counts[2])
avg_turnout = test["Turnout"].mean()
avg_vote = test["MAS.vote"].mean()
results.loc[len(results)] = [round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]

In [14]:
results.to_csv("log_tot_results.csv", index=False)
results

,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,86.73,13.27,7.53,5.74,89.8,0.46


In [15]:
col_names = ["Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
log_dep_results = pd.DataFrame(columns = col_names)
for dep in test["Department"].unique():
    vars()[dep] = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    counts = test[test["Department"] == dep].Pred_Fraud_type.value_counts()
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Department"] == dep]["Turnout"].mean()
    avg_vote = test[test["Department"] == dep]["MAS.vote"].mean()
    log_dep_results.loc[len(log_dep_results)] = [dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
log_dep_results = log_dep_results.sort_values(by = "Clean")
log_dep_results.reset_index(drop=True, inplace=True)

In [16]:
log_dep_results.to_csv("log_dep_results.csv", index=False)
log_dep_results

,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Cochabamba,78.55,21.45,15.47,5.99,90.28,0.57
1,Potosí,86.61,13.39,6.91,6.48,88.87,0.46
2,La Paz,87.38,12.62,9.89,2.74,91.75,0.53
3,Santa Cruz,88.29,11.71,8.52,3.19,88.83,0.34
4,Oruro,89.92,10.08,7.43,2.65,91.09,0.47
5,Beni,91.33,8.67,5.50,3.17,86.17,0.34
6,Chuquisaca,91.84,8.16,5.76,2.40,88.41,0.40
7,Tarija,93.37,6.63,3.71,2.92,87.26,0.39
8,Pando,93.89,6.11,4.20,1.91,84.31,0.43


In [17]:
col_names = ["Municipality", "Department", "Clean", "At_Risk", "BBS_Risk", "VS_Risk", "Avg_Turnout", "Avg_MAS_vote"]
log_municip_results = pd.DataFrame(columns = col_names)
for municip in test["Municipality"].unique():
  dep = test.loc[test["Municipality"] == municip]["Department"].iloc[0]
  counts = test[test["Municipality"] == municip].Pred_Fraud_type.value_counts()
  if (len(counts) > 2):
    clean = counts[0] / (counts[0] + counts[1] + counts[2])
    tot_risk = (counts[1] + counts[2]) / (counts[0] + counts[1] + counts[2])
    bbs = counts[1] / (counts[0] + counts[1] + counts[2])
    vs = counts[2] / (counts[0] + counts[1] + counts[2])
    avg_turnout = test[test["Municipality"] == municip]["Turnout"].mean()
    avg_vote = test[test["Municipality"] == municip]["MAS.vote"].mean()
    log_municip_results.loc[len(log_municip_results)] = [municip, dep, round(100*clean,2), round(100*tot_risk,2), round(100*bbs,2), round(100*vs,2), round(100*avg_turnout,2), round(avg_vote,2)]
log_municip_results = log_municip_results.sort_values(by = "Clean")
log_municip_results.reset_index(drop=True, inplace=True)

In [18]:
log_municip_results.to_csv("log_municip_results.csv", index=False)
log_municip_results

,Municipality,Department,Clean,At_Risk,BBS_Risk,VS_Risk,Avg_Turnout,Avg_MAS_vote
0,Chacarilla,La Paz,33.33,66.67,33.33,33.33,90.33,0.81
1,Chayanta,Potosí,43.33,56.67,40.00,16.67,91.88,0.76
2,Sacaca,Potosí,45.45,54.55,30.30,24.24,93.73,0.85
3,Papel Pampa,La Paz,45.45,54.55,36.36,18.18,90.87,0.82
4,Caripuyo,Potosí,46.15,53.85,30.77,23.08,90.48,0.84
...,...,...,...,...,...,...,...,...
123,Nuestra Señora de La Paz,La Paz,95.75,4.25,4.04,0.21,91.00,0.40
124,San Lorenzo,Tarija,95.83,4.17,2.78,1.39,87.64,0.44
125,Bermejo,Tarija,95.97,4.03,2.42,1.61,84.90,0.45
126,Mineros,Santa Cruz,96.88,3.12,1.56,1.56,88.24,0.38
